In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [4]:
#Read raw file
customers=pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement=pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing=pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transaction=pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')


## Prepare

In [14]:
customers.head()

,customer_id,join_date,last_purchase_date,age,gender,location
0,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury
1,2,2021-09-08,2023-10-25,NaN,Male,Hillville
2,3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown
3,4,2022-01-01,2022-09-01,29.0,Male,Grossstad
4,5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort


In [11]:
engagement.head()

,customer_id,number_of_site_visits,number_of_emails_opened,number_of_clicks
0,1,10,15,1
1,2,285,49,51
2,3,192,73,25
3,4,110,30,17
4,5,161,2,7


In [16]:
marketing.head()

,campaign_id,customer_id,response,promotion_type,campaign_date
0,1,1,No,Buy One Get One,2024-02-29
1,2,1,No,Discount,2024-01-24
2,3,1,No,Free Shipping,2024-03-05
3,4,1,Yes,Buy One Get One,2024-01-10
4,5,2,Yes,Free Shipping,2022-07-08


In [13]:
transaction.head()

,transaction_id,customer_id,transaction_date,transaction_amount,product_category
0,1,1,2024-02-03,165.56,Clothing
1,2,1,2024-03-02,699.01,Home Goods
2,3,1,2024-03-12,146.86,Home Goods
3,4,1,2024-01-20,927.46,Electronics
4,5,1,2024-02-25,1395.87,Electronics


In [17]:
# univariate analysis
customers.describe()

,customer_id,age
count,10000.00000,8991.000000
mean,5000.50000,43.467467
std,2886.89568,15.094380
min,1.00000,18.000000
25%,2500.75000,30.000000
50%,5000.50000,44.000000
75%,7500.25000,57.000000
max,10000.00000,69.000000


## Join in Customer level

In [19]:
marketing_agg = marketing[marketing['response']=='Yes'].groupby('customer_id')['campaign_id'].count().to_frame()


In [21]:
transactions_agg = transaction.groupby('customer_id').aggregate({'transaction_id':'count','transaction_amount':'sum'})


In [22]:
customers.set_index('customer_id', inplace=True)
engagement.set_index('customer_id', inplace=True)

In [23]:
joint_data = customers.join(engagement).join(transactions_agg).join(marketing_agg)

In [24]:
joint_data

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id
customer_id,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,1.0
2,2021-09-08,2023-10-25,NaN,Male,Hillville,285,49,51,9,6081.32,2.0
3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown,192,73,25,6,1454.87,1.0
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,1.0
5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort,161,2,7,24,15524.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,NaN
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1.0
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1.0


In [25]:
joint_data.describe()

,age,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id
count,8991.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,7665.000000
mean,43.467467,100.111900,39.862100,19.725300,12.954500,7737.562981,1.640965
std,15.094380,118.625052,46.511719,22.975083,7.191203,4628.799469,0.776240
min,18.000000,1.000000,1.000000,1.000000,1.000000,23.810000,1.000000
25%,30.000000,15.000000,6.000000,3.000000,7.000000,3839.915000,1.000000
50%,44.000000,53.000000,22.000000,11.000000,13.000000,7498.890000,1.000000
75%,57.000000,141.000000,57.000000,28.000000,19.000000,11275.797500,2.000000
max,69.000000,711.000000,303.000000,142.000000,25.000000,24298.220000,4.000000


## Clean data

### outlier

In [31]:
#find outlier：transaction_amount
joint_data[joint_data['transaction_amount'] == 24298.22]

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id
customer_id,,,,,,,,,,,
6795,2021-07-10,2022-04-28,30.0,Female,Lake Alicia,100.0,6,1,23,24298.22,2.0


In [32]:
#replace outliers 711 to 100.1119
joint_data['transaction_amount'] = joint_data['transaction_amount'].replace(24298.22, 7737.562981)

In [33]:
joint_data.describe()

,age,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,campaign_id
count,8991.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,7665.000000
mean,43.467467,100.050811,39.862100,19.725300,12.954500,7735.906915,1.640965
std,15.094380,118.467620,46.511719,22.975083,7.191203,4625.835438,0.776240
min,18.000000,1.000000,1.000000,1.000000,1.000000,23.810000,1.000000
25%,30.000000,15.000000,6.000000,3.000000,7.000000,3839.915000,1.000000
50%,44.000000,53.000000,22.000000,11.000000,13.000000,7498.890000,1.000000
75%,57.000000,141.000000,57.000000,28.000000,19.000000,11273.740000,2.000000
max,69.000000,695.000000,303.000000,142.000000,25.000000,22364.720000,4.000000


### check missing value

In [35]:
data=joint_data
missing_values = data.isnull().sum()
print(missing_values)

join_date                     0
last_purchase_date            0
age                        1009
gender                      533
location                      0
number_of_site_visits         0
number_of_emails_opened       0
number_of_clicks              0
transaction_id                0
transaction_amount            0
campaign_id                2335
dtype: int64


In [36]:
#use mean to age, unknown to gender
data['age'] = data['age'].fillna(data['age'].mean())  
data['gender'] = data['gender'].fillna('Unknown')   

In [37]:
#use 00 to present the campaign id not the target
data['campaign_id'] = data['campaign_id'].fillna(00)

In [38]:
print(data.isnull().sum())

join_date                  0
last_purchase_date         0
age                        0
gender                     0
location                   0
number_of_site_visits      0
number_of_emails_opened    0
number_of_clicks           0
transaction_id             0
transaction_amount         0
campaign_id                0
dtype: int64
